## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Acapulco,MX,16.8634,-99.8901,87.62,70,40,8.05,scattered clouds,2022-07-20 18:27:50
1,1,Ushuaia,AR,-54.8000,-68.3000,31.66,93,100,24.16,light snow,2022-07-20 18:27:50
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,64,75,24.16,broken clouds,2022-07-20 18:27:50
3,3,Dikson,RU,73.5069,80.5464,44.02,67,10,21.52,clear sky,2022-07-20 18:27:51
4,4,Tsumeb,NaN,-19.2333,17.7167,63.09,25,2,9.26,clear sky,2022-07-20 18:27:51


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Acapulco,MX,16.8634,-99.8901,87.62,70,40,8.05,scattered clouds,2022-07-20 18:27:50
6,6,Acajutla,SV,13.5928,-89.8275,87.73,57,36,7.18,scattered clouds,2022-07-20 18:27:51
7,7,Victoria,HK,22.2855,114.1577,87.33,85,24,4.00,few clouds,2022-07-20 18:24:08
11,11,Daru,PG,-9.0763,143.2092,75.94,93,66,12.12,broken clouds,2022-07-20 18:27:53
13,13,Grand Forks,US,47.9417,-97.4337,82.78,57,20,13.80,few clouds,2022-07-20 18:23:11
14,14,Jamestown,US,42.0970,-79.2353,89.19,54,75,13.80,broken clouds,2022-07-20 18:25:59
15,15,Dubti,ET,11.7361,41.0853,86.94,43,100,10.33,overcast clouds,2022-07-20 18:27:53
16,16,Saint Anthony,US,45.0205,-93.2180,88.11,41,0,25.32,clear sky,2022-07-20 18:27:54
19,19,Narasannapeta,IN,18.4167,84.0500,82.94,84,87,5.17,overcast clouds,2022-07-20 18:27:55
20,20,Atuona,PF,-9.8000,-139.0333,77.65,74,43,15.41,scattered clouds,2022-07-20 18:27:55


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
Date                   243
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Acapulco,MX,87.62,scattered clouds,16.8634,-99.8901,
6,Acajutla,SV,87.73,scattered clouds,13.5928,-89.8275,
7,Victoria,HK,87.33,few clouds,22.2855,114.1577,
11,Daru,PG,75.94,broken clouds,-9.0763,143.2092,
13,Grand Forks,US,82.78,few clouds,47.9417,-97.4337,
14,Jamestown,US,89.19,broken clouds,42.0970,-79.2353,
15,Dubti,ET,86.94,overcast clouds,11.7361,41.0853,
16,Saint Anthony,US,88.11,clear sky,45.0205,-93.2180,
19,Narasannapeta,IN,82.94,overcast clouds,18.4167,84.0500,
20,Atuona,PF,77.65,scattered clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Acapulco,MX,87.62,scattered clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
6,Acajutla,SV,87.73,scattered clouds,13.5928,-89.8275,Laureles
7,Victoria,HK,87.33,few clouds,22.2855,114.1577,Mini Central
11,Daru,PG,75.94,broken clouds,-9.0763,143.2092,Daru Lodge
13,Grand Forks,US,82.78,few clouds,47.9417,-97.4337,Warrior Inn
14,Jamestown,US,89.19,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
15,Dubti,ET,86.94,overcast clouds,11.7361,41.0853,degu Shiferaw
16,Saint Anthony,US,88.11,clear sky,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
19,Narasannapeta,IN,82.94,overcast clouds,18.4167,84.0500,AVK events and catering
20,Atuona,PF,77.65,scattered clouds,-9.8000,-139.0333,Villa Enata


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))